In [209]:
class Vektor:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __repr__(self):
        x, y = self.koordinate()
        return ('{}(x = {}, y = {})'.format(self.__class__.__name__, x, y))
    
    def __add__(self, other):
        x1 = self.x
        x2 = other.x
        y1 = self.y
        y2 = other.y
        return(Vektor(x1 + x2, y1 + y2))
    
    def __sub__(self, other):
        x1 = self.x
        x2 = other.x
        y1 = self.y
        y2 = other.y
        return(Vektor(x1 - x2, y1 - y2))
    
    def __rmul__(self, konstanta):
        x1 = konstanta * self.x
        y1 = konstanta * self.y
        return(Vektor(x1, y1))
    
    def koordinate(self):
        return self.x, self.y
    
class Premica:
    def __init__(self, u1, u2): # u tocka; rabili bomo koordinate
        u1x, u1y = u1.koordinate()
        u2x, u2y = u2.koordinate()
        if u2x - u1x != 0:
            self.koef = (u2y -u1y) / (u2x - u1x)
            self.n = u1y - self.koef * u1x
        else:
            self.koef = "NaN"
            self.x = u1x
        
    def parametri(self):
        if self.koef != "NaN":
            return self.koef, self.n
        else:
            return self.x
    
    def __repr__(self):
        if self.koef != "NaN":
            k, n = self.parametri()
            return ('{}(y = {} * x + {})'.format(self.__class__.__name__, k, n))
        else:
            x = self.parametri()
            return ('{}(x = {})'.format(self.__class__.__name__, x))
    
class U:
    def __init__(self, a, b):
        self.a = a # a konstanta ... koliko "prvih" vektorjev uporabimo
        self.b = b
        
    def __repr__(self):
        a,b = self.podatki()
        return ('{}({}a, {}b)'.format(self.__class__.__name__, a, b))
    
    def podatki(self):
        return self.a, self.b
    
    def koordinate(self):
        vektor = self.a * A1 + self.b * A2
        koordinate = vektor.koordinate()
        return [koordinate[0], koordinate[1]]

class V:
    def __init__(self, a, b):
        self.a = a # a konstanta ... koliko "prvih" vektorjev uporabimo
        self.b = b
        
    def __repr__(self):
        a,b = self.podatki()
        return ('{}({}a, {}b)'.format(self.__class__.__name__, a, b))
    
    def podatki(self):
        return self.a, self.b
    
    def koordinate(self):
        vektor = self.a * A1 + self.b * A2 + Vektor(1, 0)
        koordinate = vektor.koordinate()
        return [koordinate[0], koordinate[1]]

A1 = Vektor(0, - sqrt(3))
A2 = Vektor(3 / 2, - sqrt(3) / 2)

def grid(k, l, m, n):
    # oglisca = ([0, 0], [k, l], [m, n], [k + m, l + n])
    # print U(0,0).koordinate(), U(k, l).koordinate(), U(m, n).koordinate(), U(k + m, l + n).koordinate()
    mina = min(0, k, m, k + m)
    maxa = max(0, k, m, k + m)
    minb = min(0, l, n, l + n)
    maxb = max(0, l, n, l + n)
    
    matrika = []
    for i in range(mina, maxa + 1):
        vrstica = []
        for j in range(minb, maxb + 1):
            vrstica += [U(i, j)]
            # print U(i,j), U(i, j).koordinate()
        matrika += [vrstica]
        
    return matrika

############################### v delu ####################
def premice(k, l, m, n):
    u0 = U(0, 0)
    u1 = U(k, l)
    u2 = U(m, n)
    u3 = U(k + m, l + n)
    
    # e1, e2 sta zgornje premice, f1 in f2 pa spodnje
    if u1.koordinate()[1] >= u2.koordinate()[1]: 
        e1 = Premica(u0, u1)
        e2 = Premica(u1, u3)
        f1 = Premica(u0, u2)
        f2 = Premica(u2, u3)
    else:
        e1 = Premica(u0, u2)
        e1 = Premica(u2, u3)
        f1 = Premica(u0, u1)
        f2 = Premica(u1, u3)

    return {"premica0" : e1, "premica1" : f1, "premica2" : e2, "premica3" : f2}
        
def pogoji(tocka, premica0, premica1, premica2, premica3): 
    # premica0, premica1 sta zgornji v paralelogramu
    # premica2 je iz izhodisca v drugo smer kot premica 0
    x, y = tocka.koordinate()
    
    if (premica0.koef == "NaN"):
        # potem je tudi premica3 navpicna
        pogoj0 = (min(premica0.x, premica3.x) <= x < max(premica0.x, premica3.x))
        # premica1 je zgornja
        pogoj1 = (y <= premica1.koef * x + premica1.n)
        pogoj2 = (y > premica2.koef * x + premica2.n)
        return(pogoj0 & pogoj1 & pogoj2)
    elif (premica1.koef == "NaN"):
        # potem je tudi premica2 navpicna
        pogoj1 = (min(premica1.x, premica2.x) <= x < max(premica1.x, premica2.x))
        # premica0 je zgornja
        pogoj0 = (y <= premica0.koef * x + premica0.n)
        pogoj3 = (y > premica2.koef * x + premica2.n)
        return(pogoj0 & pogoj1 & pogoj2)
    else:
        return "NAAAAAAAAAAA"
    
from matplotlib.path import Path

def tocke_v_paralelogramu(k, l, m, n):
    G = grid(k, l, m, n)
    oglisca = [U(0,0).koordinate(), U(k, l).koordinate(), U(m, n).koordinate(), U(k + m, l + n).koordinate()]
    tocke = []
    for i in range(len(G)):
        for j in range(len(G[0])):
            tocke.append(G[i][j].koordinate())
    
    paralelogram = Path(oglisca)
    notranjost = paralelogram.contains_points(tocke)

    print(notranjost)
    
            
        
        